# Imports 

In [2]:
import pandas as pd
import numpy as np
import os #how you interact with the operating system

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Read in data and make sure relevant columns are string/character 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` (North American Industry Classification Systems)

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [4]:
# uncomment if u need to find your path: os.getcwd()
SD = sd_df = pd.read_csv("../public_data/sd_df.csv")
NA = naics_df = pd.read_csv("../public_data/naics_df.csv")


## filepaths should be easy to run bc pulling folder from git

In [6]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.dtypes
naics_df.dtypes
sd_df.head()
naics_df.head()

account_key          object
dba_name             object
council_district     object
naics_code           object
naics_description    object
naics_nchar           int64
dtype: object

naics                object
naics_description    object
dtype: object

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


## "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [133]:
## INNER JOIN

sd_df.shape #353 rows
naics_df.shape #2270 rows

inner_df = sd_df.merge(naics_df, left_on = "naics_code", right_on = "naics", how = "inner")
#sd_inner
inner_df.shape #52 rows --> only an intersection of 52



## PRINT EXAMPLES OF SD BUSINESSES LOST IN MERGE
## use anti-join logic

lost_df = sd_df[~sd_df["naics_code"].isin(inner_df["naics_code"])]
lost_df.shape
print("Examples of SD bus lost: ")
lost_df.head()


## WHY LOST?

print(sd_df["naics_nchar"].value_counts()) # see that some have 150 and then it goes down, basically the characters are not variable

# but check in the merged
print(inner_df["naics_nchar"].value_counts()) #but here you see they all only have 6 characters


(353, 7)

(2270, 2)

(52, 9)

(318, 7)

Examples of SD bus lost: 


,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar,padded_naics_code
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,541100
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,541100
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,541200
5,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4,541100
6,1981000840,BENINK & SLAVENS LLP,cd_7,5411,LEGAL SERVICES,4,541100


naics_nchar
5    150
4     89
6     49
3     42
2     23
Name: count, dtype: int64
naics_nchar
6    52
Name: count, dtype: int64


## "Left join"- retain all sd businesses even if naics code isn't in `naics_df`

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix
- Use `naics_merge_status` in the merged result to look at a sample of San Diego businesses that weren't matched with `naics_df`

In [69]:
left_df = sd_df.merge(naics_df, left_on = "naics_code", right_on = "naics", how = "left", indicator = True, suffixes = ("_sd", "_census"))
left_df.rename(columns = {"_merge":"naics_merge_status"}, inplace = True) #can also just say indicator = "naics_merge_status" within the actual merge call
left_df.head()

left_df[left_df["naics_merge_status"] == "left_only"].head()

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only


,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only
5,1979053082,GRIMM VRANJES GREER STEPHAN & BRIDGMAN LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only
6,1981000840,BENINK & SLAVENS LLP,cd_7,5411,LEGAL SERVICES,4,NaN,NaN,left_only


## Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator, `is_lost`, that equals `True` if the merge indicator is equal to "left_only" (and is otherwise false)
- Find the proportion of businesses by council district that were lost in the left join (SD businesses that failed to match to `naics_df`). To do this, group by `council_district` and use the shortcut of taking the mean of the `is_lost` indicator to find the proportions lost by `council_district`. 

In [135]:
left_df["is_lost"] = left_df["naics_merge_status"] == "left_only" #to create the binary/boolean flag column
left_df.head()

left_df.groupby("council_district").agg({"is_lost": "mean"}) #passing the function, mean

## soooo much is lost like pretty much is lost proportion is over .8 (>80% is lost) for all of them

left_df.is_lost.mean() #to see on average --> bc True and False correspond to 0 and 1 (binary)-->this works on that column

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,naics_merge_status,is_lost
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both,False
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only,True
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6,541211,Offices of Certified Public Accountants,both,False
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4,NaN,NaN,left_only,True
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4,NaN,NaN,left_only,True


,is_lost
council_district,
cd_1,0.880597
cd_2,0.804348
cd_3,0.812500
cd_4,1.000000
cd_5,0.916667
cd_6,0.859649
cd_7,0.918919
cd_8,0.933333
cd_9,1.000000


0.8594594594594595

## Optional challenge exercise: add lagging 0's and see if merge rate improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were often less than six digits long, while the Census ones were always 6 digits.

You wonder if this is an issue where 0's in some of the SD's data naics codes got cut off (eg 540000 became 54), and if so whether adding these lagging zeros would improve the merge rate in a left join.

- Pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits, using one of two approaches: 
  1. `str.pad` in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html)
  2. for more of a challenge, write your own function! It should check the # of digits in the naics code string and pad it with 0's at the end up to 6 digits. To execute your function, use row-wise apply: `df.apply(lambda row: funcname(row.column), axis=1)`.
- Perform the same left join as above and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [141]:
padded_sd = sd_df['naics_code'].astype(str).str.pad(6, side='right', fillchar='0')

sd_df["padded_naics_code"] = padded_sd

#sd_df.head()

left_challenge_df = sd_df.merge(naics_df, left_on = "padded_naics_code", right_on = "naics", how = "left", indicator = True, suffixes = ("_sd", "_census"))
left_challenge_df.rename(columns = {"_merge":"naics_merge_status"}, inplace = True)
#left_challenge_df.head()

left_challenge_df["is_lost"] = left_challenge_df["naics_merge_status"] == "left_only"
#left_challenge_df.head()


## COMPARING (with last match?)
left_df.groupby("council_district").agg({"is_lost": "mean"})
left_challenge_df.groupby("council_district").agg({"is_lost": "mean"}) 

left_df.is_lost.mean() #0.8594594594594595
left_challenge_df.is_lost.mean() #0.46860986547085204 --> see how much less is lost on average 
## compare the naics_descrption from SD vs census in the new dataset for a sample of these new matches

left_challenge_df[["naics_description_sd", "naics_description_census"]]
left_df[["naics_description_sd", "naics_description_census"]]

,is_lost
council_district,
cd_1,0.880597
cd_2,0.804348
cd_3,0.812500
cd_4,1.000000
cd_5,0.916667
cd_6,0.859649
cd_7,0.918919
cd_8,0.933333
cd_9,1.000000


,is_lost
council_district,
cd_1,0.546667
cd_2,0.555556
cd_3,0.348148
cd_4,1.000000
cd_5,0.483871
cd_6,0.445946
cd_7,0.560976
cd_8,0.333333
cd_9,1.000000


0.8594594594594595

0.46860986547085204

,naics_description_sd,naics_description_census
0,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,Offices of Certified Public Accountants
1,LEGAL SERVICES,NaN
2,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,Offices of Certified Public Accountants
3,LEGAL SERVICES,NaN
4,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,NaN
...,...,...
441,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",NaN
442,ALL OTHER AUTOMOTIVE R&M,All Other Automotive Repair and Maintenance
443,ALL OTHER AUTOMOTIVE R&M,All Other Automotive Repair and Maintenance
444,ALL OTHER AUTOMOTIVE R&M,All Other Automotive Repair and Maintenance


,naics_description_sd,naics_description_census
0,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,Offices of Certified Public Accountants
1,LEGAL SERVICES,NaN
2,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,Offices of Certified Public Accountants
3,LEGAL SERVICES,NaN
4,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,NaN
...,...,...
365,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",NaN
366,ALL OTHER AUTOMOTIVE R&M,All Other Automotive Repair and Maintenance
367,ALL OTHER AUTOMOTIVE R&M,All Other Automotive Repair and Maintenance
368,ALL OTHER AUTOMOTIVE R&M,All Other Automotive Repair and Maintenance


In [149]:
## to do with a function
pad_zeros = lambda x: "NAICS: "+ x + "0"*(6-len(x))

pad_zeros("456")

SD.naics_code.apply(pad_zeros)

'NAICS: 456000'

0      NAICS: 541211
1      NAICS: 541100
2      NAICS: 541211
3      NAICS: 541100
4      NAICS: 541200
           ...      
348    NAICS: 541000
349    NAICS: 541000
350    NAICS: 541000
351    NAICS: 811198
352    NAICS: 531000
Name: naics_code, Length: 353, dtype: object